# Day 15: Beacon Exclusion Zone
A distress signal is received. A system of sensors is deployed. None of the sensors is locked on the beacon sending the distress signal. In order to narrow the search, eliminate the areas where the source couldn't be. For further instructions: https://adventofcode.com/2022/day/15

To use example input, enter "example = True" in the following cell, or enter "example = False" to use puzzle input. Then restart and run all cells.

In [1]:
example = False

<hr>

In [2]:
import pandas as pd
import numpy as np

Read the puzzle input, or example input, according to the user's choice.

In [3]:
def read_input(example):

    if example:
        puzzle = '''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3'''
    
    else:
        puzzle = '''Sensor at x=2899860, y=3122031: closest beacon is at x=2701269, y=3542780
Sensor at x=1836719, y=1116779: closest beacon is at x=2037055, y=2000000
Sensor at x=3995802, y=2706630: closest beacon is at x=3944538, y=3053285
Sensor at x=2591204, y=2008272: closest beacon is at x=2597837, y=2509170
Sensor at x=2546593, y=1538222: closest beacon is at x=2037055, y=2000000
Sensor at x=252214, y=61954: closest beacon is at x=1087565, y=-690123
Sensor at x=950, y=1106672: closest beacon is at x=-893678, y=1276864
Sensor at x=1349445, y=1752783: closest beacon is at x=2037055, y=2000000
Sensor at x=3195828, y=3483667: closest beacon is at x=3334657, y=3531523
Sensor at x=2057761, y=2154359: closest beacon is at x=2037055, y=2000000
Sensor at x=2315350, y=3364640: closest beacon is at x=2701269, y=3542780
Sensor at x=327139, y=2426600: closest beacon is at x=-88420, y=3646947
Sensor at x=3943522, y=2854345: closest beacon is at x=3944538, y=3053285
Sensor at x=3358620, y=516881: closest beacon is at x=3260516, y=2246079
Sensor at x=1788376, y=8679: closest beacon is at x=1087565, y=-690123
Sensor at x=3344883, y=3537985: closest beacon is at x=3334657, y=3531523
Sensor at x=2961064, y=2697125: closest beacon is at x=2597837, y=2509170
Sensor at x=3780090, y=2093546: closest beacon is at x=3260516, y=2246079
Sensor at x=3291917, y=3398703: closest beacon is at x=3334657, y=3531523
Sensor at x=3999864, y=2998005: closest beacon is at x=3944538, y=3053285
Sensor at x=2919272, y=3732950: closest beacon is at x=2701269, y=3542780
Sensor at x=2057404, y=2947435: closest beacon is at x=2037055, y=2000000
Sensor at x=1072126, y=645784: closest beacon is at x=1087565, y=-690123
Sensor at x=3549465, y=2554712: closest beacon is at x=3260516, y=2246079
Sensor at x=3550313, y=3121694: closest beacon is at x=3944538, y=3053285
Sensor at x=3405149, y=3483630: closest beacon is at x=3334657, y=3531523
Sensor at x=2600212, y=3961193: closest beacon is at x=2701269, y=3542780
Sensor at x=1102632, y=3932527: closest beacon is at x=-88420, y=3646947
Sensor at x=67001, y=3506079: closest beacon is at x=-88420, y=3646947
Sensor at x=3994250, y=3975025: closest beacon is at x=3944538, y=3053285
Sensor at x=3019750, y=2125144: closest beacon is at x=3260516, y=2246079
Sensor at x=3282319, y=3656404: closest beacon is at x=3334657, y=3531523
Sensor at x=2797371, y=3645126: closest beacon is at x=2701269, y=3542780'''
        
    return puzzle


puzzle = read_input(example)


Parse the coordinates of the sensors and beacons into a pandas dataframe, s. Calculate the exclusion distance, d_ex, as the Manhattan distance from each sensor to its closest beacon. No other beacon can be closer.

In [4]:
def parse_sensor_data(puzzle):
    
    sensors = puzzle.split('\n')
    table = []

    for sensor in sensors:
        row = []
        field = sensor.split()
        row.append(int(field[2][2:-1]))
        row.append(int(field[3][2:-1]))
        row.append(int(field[8][2:-1]))
        row.append(int(field[9][2:]))
        table.append(row)

    sens = pd.DataFrame(table, columns=['x_s', 'y_s', 'x_b', 'y_b'])

    sens['d_ex'] = abs(sens['x_b'] - sens['x_s']) + abs(sens['y_b'] - sens['y_s'])
    
    return sens


sens = parse_sensor_data(puzzle)
sens.head()


,x_s,y_s,x_b,y_b,d_ex
0,2899860,3122031,2701269,3542780,619340
1,1836719,1116779,2037055,2000000,1083557
2,3995802,2706630,3944538,3053285,397919
3,2591204,2008272,2597837,2509170,507531
4,2546593,1538222,2037055,2000000,971316


Consider a single row, y = n. The shortest distance to the row from each sensor is |y_s - n|. We'll call this distance d_row. 

Select only the rows for nearby sensors, 'sensn', for which the distance to row n is smaller than d_ex. The other sensors are too far from this row to exclude any locations.

Call the remaining distance d_rem. This distance can be applied along the row in either direction, toward x_min or x_max. 

In [5]:
def find_xmin_xmax(sens, n):
    sens['d_row'] = abs(sens.y_s - n)

    nearsens = sens.copy()
    nearsens = nearsens[nearsens.d_row < nearsens.d_ex]

    nearsens['d_rem'] = nearsens.d_ex - nearsens.d_row

    nearsens['x_min'] = nearsens.x_s - nearsens.d_rem
    nearsens['x_max'] = nearsens.x_s + nearsens.d_rem
    
    return nearsens


if example:
    n = 10
else:
    n = 2000000

nearsens = find_xmin_xmax(sens, n)

nearsens


,x_s,y_s,x_b,y_b,d_ex,d_row,d_rem,x_min,x_max
1,1836719,1116779,2037055,2000000,1083557,883221,200336,1636383,2037055
3,2591204,2008272,2597837,2509170,507531,8272,499259,2091945,3090463
4,2546593,1538222,2037055,2000000,971316,461778,509538,2037055,3056131
6,950,1106672,-893678,1276864,1064820,893328,171492,-170542,172442
7,1349445,1752783,2037055,2000000,934827,247217,687610,661835,2037055
9,2057761,2154359,2037055,2000000,175065,154359,20706,2037055,2078467
11,327139,2426600,-88420,3646947,1635906,426600,1209306,-882167,1536445
13,3358620,516881,3260516,2246079,1827302,1483119,344183,3014437,3702803
17,3780090,2093546,3260516,2246079,672107,93546,578561,3201529,4358651
21,2057404,2947435,2037055,2000000,967784,947435,20349,2037055,2077753


The exclusion zone for each sensor is the set of locations ranging from x_min to x_max.

In [6]:
nearsens['ex_zone'] = nearsens.apply(lambda r: set(range(r.x_min, r.x_max + 1)), axis=1)
if example:
    print(nearsens[['x_s', 'y_s', 'd_ex', 'x_min', 'x_max', 'ex_zone']])

The set of all excluded locations is the union of these sets. No *unknown* beacon can be in any of these locations. 

In [7]:
excluded = set({})

for zone in nearsens.ex_zone:
    excluded = excluded.union(zone)
    
if example:
    print(excluded)

There is, however, the possibility that a *known* beacon is located within the exclusion zone. These locations need to be subtracted from the excluded locations to find the locations where a beacon cannot be.

In [8]:
nearsens[(nearsens.y_b == n) & ~nearsens[['x_b', 'y_b']].duplicated()]

,x_s,y_s,x_b,y_b,d_ex,d_row,d_rem,x_min,x_max,ex_zone
1,1836719,1116779,2037055,2000000,1083557,883221,200336,1636383,2037055,"{1636383, 1636384, 1636385, 1636386, 1636387, ..."


In [9]:
print('Excluded locations:', len(excluded) - len(nearsens[(nearsens.y_b == n) & ~nearsens[['x_b', 'y_b']].duplicated()]))

Excluded locations: 5240818


## Part 2

Coordinate limits are defined, and now we will search for the source of the distress signal. The instructions state there's only one possible position.

We could iterate through the rows for the example data. But we need a solution that will also work for the puzzle data. So first, let's consider whether we can reasonably loop through 4 million rows, if each row takes about 5 seconds for my puzzle input as observed above.

In [10]:
5 * 4000000 / 60 / 60 / 24 / 30

7.71604938271605

Iterating through rows will take almost 8 months. We don't have that kind of time. 

If there's only one point where the distress beacon can be, then it must be completely surrounded by exclusion zones and/or coordinate boundaries. We'll restrict our search to the spaces that are one step beyond the exclusion zones of the sensors, and within the coordinate boundaries. 

The exclusion zone of a sensor is a square, extending from the sensor to +/- d_ex in both x and y.  

In [11]:
sens = parse_sensor_data(puzzle)
sens['x_min'] = sens.x_s - sens.d_ex
sens['x_max'] = sens.x_s + sens.d_ex
sens['y_min'] = sens.y_s - sens.d_ex
sens['y_max'] = sens.y_s + sens.d_ex

sens = sens.sort_values(by=['d_ex'])
sens = sens.reset_index()

Search just outside the perimeter of each exclusion zone.

In [12]:
def search_perimeter(sensor):
    '''Return all points, within coordinate limits, that are just outside a sensor's exclusion zone.
    w2s (west to south) goes from (xmin-1, ys) to (xs, ymax+1), and so on.'''
    x_s = sensor.x_s
    y_s = sensor.y_s
    d_ex = sensor.d_ex
    w2s = set([(x_s - (d_ex + 1) + x, y_s + x) for x in range((d_ex + 1) + 1)])
    s2e = set([(x_s + x, y_s + (d_ex + 1) - x) for x in range((d_ex + 1) + 1)])
    e2n = set([(x_s + (d_ex + 1) - x, y_s - x) for x in range((d_ex + 1) + 1)])
    n2w = set([(x_s - x, y_s - (d_ex + 1) + x) for x in range((d_ex + 1) + 1)])
    points = w2s.union(s2e, e2n, n2w)
    return points

In [13]:
%time sens['p_plus'] = sens.apply(search_perimeter, axis=1)

Wall time: 1min 22s


Filter out points outside the coordinate boundary.

In [17]:
if example:
    c_max = 20
else:
    c_max = 4000000

        
def in_bounds(point):
    '''True where point (x,y) is within coordinate boundaries'''
    return (point[0] > 0 and point[0] <= c_max and
            point[1] > 0 and point[1] <= c_max)

In [18]:
%time sens.p_plus = sens.p_plus.apply(lambda x: set(filter(in_bounds, x)))

Wall time: 1min 27s


Now look for points that are adjacent to 3 exclusion zones.

In [38]:
bounded_3 = set({})

for i in range(len(sens)):
    s1 = sens.loc[i]
    for j in range(i+1, len(sens)):
        s2 = sens.loc[j]
        for k in range(j+1, len(sens)):
            s3 = sens.loc[k]
            hope = s1.p_plus.intersection(s2.p_plus, s3.p_plus)
            if len(hope) > 0:
                bounded_3 = bounded_3.union(hope)

                    
bounded_3

{(1882696, 2154360),
 (2037054, 2000000),
 (2037055, 2000001),
 (2057405, 1979651),
 (2057762, 1979294),
 (2701268, 3542780),
 (2701269, 3542779),
 (3260515, 2246079),
 (3260516, 2246080),
 (3303271, 2906101),
 (3328195, 3537986),
 (3334657, 3531524),
 (3334658, 3531523),
 (3344884, 3521297),
 (3944537, 3053285),
 (3944538, 3053286)}

Now remove points that are in any sensor's exclusion zone.

In [39]:
def excluded(point, sensor):
    return abs(point[0] - sensor.x_s) + abs(point[1] - sensor.y_s) <= sensor.d_ex
    

excluded_points = set({})

for point in bounded_3:
    for s in range(len(sens)):
        sensor = sens.loc[s]        
        if excluded(point, sensor):
            excluded_points.add(point)
            

In [40]:
answer = bounded_3.difference(excluded_points)
answer

{(3303271, 2906101)}

In [41]:
for point in answer:
    print('Tuning frequency is:', 4000000 * point[0] + point[1])

Tuning frequency is: 13213086906101
